In [ ]:
import pandas as pd
from src.analyze.analyze_punctuality import *
from src.analyze.analyze_avg_speed import combine_bus_locations_within_hour, group_by_bus
from visualisation.common import *

To analyze buses punctuality, first we will take a look at a tables that we will be working with. 

In [ ]:
df_bus_stop_info_bus_stops_coord = pd.read_json(bus_stops_coordinates)
df_timetables = pd.read_json(timetables)

print(df_bus_stop_info_bus_stops_coord.head())
print(df_timetables.head())

In [ ]:
df_bus_stop_info = pd.merge(df_bus_stop_info_bus_stops_coord, df_timetables, left_on=["zespol", "slupek"], right_on=["busstopId", "busstopNr"])
df_bus_stop_info = df_bus_stop_info.drop(columns=["id_ulicy", "zespol", "nazwa_zespolu", "busstopNr"])
print(df_bus_stop_info)

We will take a quick look at table's shape to see how it looks like.

In [ ]:
print(df_bus_stop_info.shape)

Let's again take a look at location's table.

In [ ]:
df_locations = combine_bus_locations_within_hour(buses_locations_at_8)
df_locations = group_by_bus(df_locations)
print(df_locations)

In [ ]:
def analyze_punctuality_thread_executor(row, bus_stop_timetables, lat, lon):
    with ThreadPoolExecutor() as executor:
        result = executor.submit(analyze_punctuality_for_a_bus_stop, df_locations, row, bus_stop_timetables, lat, lon)
        return result.result()

df_bus_stop_info['delay'] = df_bus_stop_info.head(400).apply(lambda row: analyze_punctuality_thread_executor(row, 'rozklad', 'szer_geo', 'dlug_geo'), axis=1)

df_bus_stop_info = df_bus_stop_info.dropna(subset=['delay'], axis=0)
df_bus_stop_info = df_bus_stop_info[df_bus_stop_info['delay'] != -1]

print(df_bus_stop_info.head(55)['delay'])


Now we will see the mean value of  (in minutes) of buses on x bus stops. 

In [ ]:
print(df_bus_stop_info['delay'].mean())